# **CRDM Beta Simulation (CCB Project): Version 4**

#### Softmax vs Luce Choice Models
#### - Alpha [0.3, 1.2, 0.05] vs alpha constant [0.6]
#### - Beta [-1.3, 1.31, 2.6/498], obtaining 500 steps (*simulated subjects*)
#### - Gamma [0.5, 8, 0.5] | Mu [0.5, 0.7, 0.01]

##### **Version 4 Improvements** -- Simulates data across stochasticity parameters (gamma/mu)

In [6]:
"""
===================
Mandy Renfro (2024)
===================
"""

from glob import glob
import numpy as np
np.seterr(all = "ignore") #disable numpy error prints in output
import os
import os.path, sys
import pandas as pd
import pylab as plt
import seaborn as sns
sns.set_theme(style = "white", palette = "muted")
from scipy.stats import linregress, shapiro, spearmanr
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

base_proj_dir = "Z:/data/CCB" ## base project directory
save_dir = os.path.join(base_proj_dir, "derivatives/simulations/crdm/simulation4") ## save directory for CRDM simulations
if not os.path.exists(save_dir): ## new Ss
    os.makedirs(save_dir) ## make new Ss save directory

def pd_extend(df_dest, df_extending):
    """ Concatenates all subjects dataframe for saving current Ss parameters
        INPUT:
        - df_dest: destination dataframe to which second dataframe will be appended
        - df_extending: dataframe to be added to the destination dataframe
        OUTPUT:
        - Concatenated dataframe containing info from currently iterated Ss
    """
    ## concatenates dataframe for saving simulated data
    return pd.concat([df_dest, df_extending])

def SVs(alpha, beta, lottery_value, certain_value, ambiguity, probability):
    """ Calculate SV for lottery and safe options given the current trial condition combination (Gilboa & Schmeidler, 1989)
        *Note: np.sign() and np.abs() allows this function to flexibly handle *both* gain and loss trials
        INPUT:
        - alpha: current Ss alpha
        - beta: current Ss beta
        - lottery_value: winning lottery amount for current condition combo
        - ambiguity: ambiguity level for current condition combo
        - probability: probability for current condition combo
        OUTPUT:
        - SV_lottery: subjective value for lottery option for current trial condition combination
        - SV_certain: subjective value for certain option fr current trial condition combination
    """
    SV_lottery = (probability - beta * ambiguity / 2) * lottery_value**alpha
    SV_certain = certain_value**alpha
    return SV_lottery, SV_certain

In [ ]:
n_trials_per_subgroup = 2 ## number of trials per probability/lottery combo

lottery_range = [8, 25, 40, 50] ## possible winning lottery values
certain_value = 5 ## value of safe option
## setting up variable arrays for parallel computations
ambiguities_extend   = np.array([0.24,  0.5,  0.74,     0,    0,    0,    0,    0]) ## ambiguities for all amb/prob pairing
probabilities_extend = np.array([ 0.5,  0.5,   0.5,  0.13, 0.25, 0.38, 0.50, 0.75]) ## probabilties for all amb/prob pairing
lotteries = np.array([])
ambiguities = np.array([])
probabilities = np.array([])
extend_len = len(ambiguities_extend) ## length of times to duplicate each lottery value
for i, lott in enumerate(lottery_range): ## for each of the four possible lottery values
    lotteries = np.hstack((lotteries, np.ones(extend_len) * lott)) ## adds current lott value in quantity of ambiguities_extend (8)
    ambiguities = np.hstack((ambiguities, ambiguities_extend)) ## adds ambiguity_extend as many times as lottery_range is long (4)
    probabilities = np.hstack((probabilities, probabilities_extend)) ## adds probabilities_extend as many times as lottery_range is long (4)
total_trials = n_trials_per_subgroup * len(lotteries) ## total number of trials per pain condition
print(total_trials)

## **Softmax Choice Model**

In [ ]:
def binomial_likelihood_SM(alpha, beta, gamma, y, n,  lottery_value, certain_value, ambiguity, probability):
    """ Calculates binomial LL for each parameter combination
        *See above notes for why we can use our own math rather than rely on bernoulli.logpmf()
        *Deals with positive infinity values by assigning a value of 0. Then the MLE functions, 
        parameter pairs with a 0 LL score are removed.
        INPUT:
        - alpha: current Ss risk parameter [high values indicate risk seeking]
        - beta: current Ss ambiguity parameter [high values indicate ambiguity avoidance]
        - gamma: current Ss choice stochasticity parameter [high values indicate more noise]
        - y: summation of lottery choices in condition subgroup
        - n: number of trials per condition combo
        - lottery_value: current condition combo winning lottery amount 
        - certain_value: current condition combo certain amount
        - ambiguity: current condition combo ambiguity level
        - probability: current condition combo probability level
        OUTPUT:
        - log_likelihood: relative score representing probabilty data resulted from current parameter combo
    """
    p = probability_of_lottery_choice_SM(alpha, beta, gamma, lottery_value, certain_value, ambiguity, probability)
    log_likelihood = (np.log(p[0]) * y + np.log(1 - p[0]) * (n - y))
    log_likelihood[log_likelihood == np.inf] = 0
    return log_likelihood

def make_data_and_recover_SM(p_alpha, p_beta, p_gamma, pid, df):
    """ Executes parameter estimation and saves results to df_participants dataframe.
        INPUT:
        - p_alpha: current simulation Ss alpha value
        - p_beta: current simulation Ss beta value
        - pid: current simulation Ss subject ID
        - df: Ss dataframe with all previous Ss information (adds by extending current Ss df)
        OUPUT:
        - *res: recovered alpha, recovered beta, recovered gamma
        - df: extended df_participants dataframe
    """
    pairs = []  
    all_data = None 
    for i, amb in enumerate(ambiguities):
        subset_trials = np.random.random(size = n_trials_per_subgroup) ## simulated randomness of choice
        p_true = probability_of_lottery_choice_SM(p_alpha, p_beta, p_gamma, lotteries[i], certain_value, amb, probabilities[i])
        idx = np.where(subset_trials <= p_true[0])
        sv_lott = p_true[1]
        sv_safe = p_true[2]
        subset_trials[:] = 0
        subset_trials[idx] = 1 ## error prone, real trials
        #subset_trials[:int(np.round(n_trials_per_subgroup*p_true))] = 1 ## perfect circumstances, no noise
        ## save data to df
        data = [pid, p_alpha, p_beta, p_gamma, None, None, None, lotteries[i], certain_value, amb, probabilities[i], sv_lott, sv_safe]
        data = [[a] * n_trials_per_subgroup for a in data]
        data.append(subset_trials)
        data = np.array(data).T
        if all_data is None:
            all_data = data
        else:
            all_data = np.vstack((all_data, data))
        lottery_chosen_x = np.sum(subset_trials)
        pairs.append((lottery_chosen_x, n_trials_per_subgroup))
    res = MLE_alpha_gamma_SM(np.array(pairs), lotteries, certain_value, ambiguities, probabilities)
    res = MLE_beta_SM(np.array(pairs), res[0], res[1], lotteries, certain_value, ambiguities, probabilities)
    df = pd_extend(df, pd.DataFrame(all_data, columns = df.columns))
    idx = np.where(df["Recovered Alpha"].values == None)
    df["Recovered Alpha"].values[idx] = res[0]
    df["Recovered Beta"].values[idx] = res[1]
    df["Recovered Gamma"].values[idx] = res[2]
    return *res, df

def MLE_alpha_gamma_SM(pairs, lotteries, certain_value, ambiguities, probabilities):
    """ Grid search of alpha and gamma parameter space to determine which point produces maximum log likelihood score. 
        *Risk trials only*
        *Notes: Conceptualize parameter and probability space as two parallel multidimensional spaces. 
                Using a "grid search" method, we carefully iterate through a 2D parameter space (alpha/gamma) 
                to determine which point best explains the data, quantified as the maximum likelihood score.
        INPUT:
        ** Numpy parallel arrays **
        - pairs: Ss choices for all condition subgroups
        - lotteries: winning lottery amounts for conditon subgroups
        - certain_values: certain amounts for condition subgroups
        - ambiguities: ambiguity level for condition subgroups
        - probabilities: probability level for condition subgroups
        OUTPUT:
        - Tuple containing best fit parameters (alpha and gamma)
    """
    best_fit = None
    max_likelihood = None
    y, n = pairs[:, 0], pairs[:, 1]
    idx = np.where(ambiguities == 0)
    for alpha in np.round(np.arange(    0.3, 2, 0.01), 2): ## grid search - part 1 (fit alpha first)
        for gamma in np.round(np.arange(0.5, 8, 0.10), 2):
            ## sums log likelihoods of all condition subgroups (beta held constant at 0, as is ambiguity level)
            likelihood = np.nansum(binomial_likelihood_SM(alpha, 0, gamma, y[idx], n[idx], lotteries[idx], certain_value, ambiguities[idx], probabilities[idx]))
            if max_likelihood is None or likelihood > max_likelihood:
                max_likelihood = likelihood
                best_fit = (alpha, gamma)
    return best_fit

def MLE_beta_SM(pairs, alpha, gamma, lotteries, certain_value, ambiguities, probabilities):
    """ Grid search of beta parameter space to determine which point produces maximum log likelihood score. 
        *Ambiguity trials only*
        *Notes: Conceptualize parameter and probability space as two parallel multidimensional spaces. 
        Using a "grid search" method previously to determine the best fit of alpha and gamma parameters,
        we can use those values to now guide our search through 1D beta space.
        INPUT:
        ** Numpy parallel arrays **
        - pairs: Ss choices for all condition subgroups
        - lotteries: winning lottery amounts for conditon subgroups
        - certain_values: certain amounts for condition subgroups
        - ambiguities: ambiguity level for condition subgroups
        - probabilities: probability level for condition subgroups
        OUTPUT:
        - Tuple containing best fit parameters (alpha, beta, and gamma)
    """
    best_fit = None
    max_likelihood = None
    y, n = pairs[:, 0], pairs[:, 1]
    idx = np.where(ambiguities != 0)
    for beta in np.round(np.arange(-1.3, 1.31, 0.01), 2): ## grid search - part 2 (fit beta second)
        ## sums log likelihoods of all condition subgroups (trials where ambiguity isn't 0)
        likelihood = np.nansum(binomial_likelihood_SM(alpha, beta, gamma, y[idx], n[idx], lotteries[idx], certain_value, ambiguities[idx], probabilities[idx]))
        if max_likelihood is None or likelihood > max_likelihood:
            max_likelihood = likelihood
            best_fit = (alpha, beta, gamma)
    return best_fit

def probability_of_lottery_choice_SM(alpha, beta, gamma, lottery_value, certain_value, ambiguity, probability):
    """ Determines probability of selecting lottery using the Softmax probabilitic function
        INPUT:
        - alpha: current Ss risk parameter [high values indicate risk seeking]
        - beta: current Ss ambiguity parameter [high values indicate ambiguity avoidance]
        - gamma: current Ss choice stochasticity parameter [high values indicate more noise]
        - lottery_value: current condition combo winning lottery amount 
        - certain_value: current condition combo certain amount
        - ambiguity: current condition combo ambiguity level
        - probability: current condition combo probability level
        OUTPUT:
        - Ss probability of choosing lottery for trials with the current condition combo
        - SV_lottery: SV for current lottery option
        - SV_certain: SV for current safe option
    """
    SV_lottery, SV_certain = SVs(alpha, beta, lottery_value, certain_value, ambiguity, probability)
    return 1 / (1 + np.exp(-gamma * (SV_lottery - SV_certain))), SV_lottery, SV_certain

### **Softmax #1**
- Simulating across beta and gamma

In [ ]:
## Participant parameter space (~7,500 simulated subs)
alphas = np.array([0.6]) ## 1 step
betas  = np.round(np.arange(-1.3, 1.31, 2.6/498), 3) ## 500 steps
gammas = np.round(np.arange( 0.5,    8,     0.5), 2) ## 16 steps
print("Number of Simulations:", len(alphas)*len(betas)*len(gammas))
df = pd.DataFrame()

df_participants = pd.DataFrame(columns=["PID", 
                                        "Simulated Alpha", "Simulated Beta", "Simulated Gamma", 
                                        "Recovered Alpha", "Recovered Beta", "Recovered Gamma",
                                        "Lottery Value", "Safe Value", 
                                        "Ambiguity", "Probability", 
                                        "SV Lottery", "SV Safe",
                                        "Choice"])
recovered_betas = []
pid = 1
for p_gamma in gammas:
    for p_alpha in alphas:
        for p_beta in betas:
            r_alpha, r_beta, r_gamma, df_participants = make_data_and_recover_SM(p_alpha, p_beta, p_gamma, pid, df_participants)
            recovered_betas.append(r_beta)
            print(pid, end = "\r")
            pid += 1  
filename = os.path.join(save_dir, "simulation4SM_cAvBvG-2trials.csv")
df_participants.to_csv(filename)

beta_x = betas.tolist() * (alphas.shape[0] * gammas.shape[0])
fit_line = linregress(beta_x, recovered_betas)
fit_stat = spearmanr(beta_x, recovered_betas)
line_y = np.array(beta_x) * fit_line.slope + fit_line.intercept
residuals = np.array(beta_x) - np.array(recovered_betas)
print("Shapiro-Wilk Test of Normality:", shapiro(np.abs(residuals))[:])
print("Spearmans Rho^2:", fit_stat[:])
print("Residual SD:", np.std(residuals))
df["Simulated β (Softmax)"] = beta_x
df["Fit β"] = recovered_betas
fig, ax = plt.subplots(figsize=(10, 10))
ax = sns.regplot(x = "Simulated β (Softmax)", y = "Fit β", data = df, color = "#26B64A")
sns.despine(top = True)
ax.set_ylabel("Fit β", fontsize=16)
ax.set_xlabel("Simulated β (Softmax)", fontsize=16)
plt.suptitle("n={5} (α=0.6, ɣ=[0.5,8,0.5]) | Shapiro-Wilk={0:.2f}, p={1:.2f}** | Spearmans Rho^2={2:.2f}, p={3:.2f}** | Residual SD={4:.2f}".format(shapiro(np.abs(residuals))[0], 
                                                                                                                                                    shapiro(np.abs(residuals))[1], 
                                                                                                                                                    fit_stat[0]**2, fit_stat[1], 
                                                                                                                                                    np.std(residuals), 
                                                                                                                                                    len(beta_x)), 
                                                                                                                                                    fontsize = 12)
fig_name = os.path.join(save_dir, "simulation4SM_cAvBvG-2trials.png")
print("Saving to: {}".format(fig_name))
plt.savefig(fig_name)

### **Softmax #2** 
- Simulating across alpha, beta, and gamma

In [ ]:
## Participant parameter space (~14,310 simulated subs)
alphas = np.round(np.arange( 0.3,   1.2,  0.05), 2)
betas  = np.round(np.arange(-1.3,  1.31,  0.05), 3) ## 52 steps
gammas = np.round(np.arange( 0.5,     8,   0.5), 2) ## 16 steps
print("Number of Simulations:", len(alphas)*len(betas)*len(gammas))
df = pd.DataFrame()

df_participants = pd.DataFrame(columns=["PID", 
                                        "Simulated Alpha", "Simulated Beta", "Simulated Gamma", 
                                        "Recovered Alpha", "Recovered Beta", "Recovered Gamma",
                                        "Lottery Value", "Safe Value", 
                                        "Ambiguity", "Probability",
                                        "SV Lottery", "SV Safe",
                                        "Choice"])
pid = 1
for p_gamma in gammas:
    for p_alpha in alphas:
        for p_beta in betas:
            r_alpha, r_beta, r_gamma, df_participants = make_data_and_recover_SM(p_alpha, p_beta, p_gamma, pid, df_participants)
            print(pid, end = "\r")
            pid += 1
filename = os.path.join(save_dir, "simulation4SM_vAvBvG-2trials.csv")
df_participants.to_csv(filename)

In [ ]:
file = glob(os.path.join(base_proj_dir, save_dir, "simulation4SM_vAvBvG-2trials.csv"))[0]
df = pd.read_csv(file)
df2 = pd.DataFrame()

beta_x = df["Simulated Beta"].values[::total_trials]
fit_line = linregress(beta_x, df["Recovered Beta"].values[::total_trials])
fit_stat = spearmanr(beta_x, df["Recovered Beta"].values[::total_trials]) ## non-normal distribution
line_y = np.array(beta_x) * fit_line.slope + fit_line.intercept
residuals = np.array(beta_x) - df["Recovered Beta"].values[::total_trials]
print("Shapiro-Wilk Test of Normality:", shapiro(np.abs(residuals))[:])
print("Spearmans Rho^2:", fit_stat[:])
print("Residual SD:", np.std(residuals))
df2["Simulated β (Softmax)"] = df["Simulated Beta"].values[::total_trials]
df2["Fit β"] = df["Recovered Beta"].values[::total_trials]
fig, ax = plt.subplots(figsize=(10, 10))
ax = sns.regplot(x = "Simulated β (Softmax)", y = "Fit β", data = df2)
sns.despine(top = True)
ax.set_ylabel("Fit β", fontsize=16)
ax.set_xlabel("Simulated β (Softmax)", fontsize=16)
plt.suptitle("n={5} (α[0.3,1.2,0.05], ɣ=[0.5,8,0.5]) | Shapiro-Wilk={0:.2f}, p={1:.2f}** | Spearmans Rho^2={2:.2f}, p={3:.2f}** | Residual SD={4:.2f}".format(shapiro(np.abs(residuals))[0], 
                                                                                                                                                            shapiro(np.abs(residuals))[1], 
                                                                                                                                                            fit_stat[0], 
                                                                                                                                                            fit_stat[1], 
                                                                                                                                                            np.std(residuals), 
                                                                                                                                                            len(beta_x)), 
                                                                                                                                                            fontsize = 12)
fig_name = os.path.join(save_dir, "simulation4SM_vAvBvG-2trials.png")
print("Saving to: {}".format(fig_name))
plt.savefig(fig_name)

## **Luce Choice Model**

In [ ]:
def binomial_likelihood_LUCE(alpha, beta, mu, y, n, lottery_value, certain_value, ambiguity, probability):
    p = probability_of_lottery_choice_LUCE(alpha, beta, mu, lottery_value, certain_value, ambiguity, probability)
    log_likelihood = (np.log(p[0]) * y + np.log(1 - p[0]) * (n - y))
    log_likelihood[log_likelihood == np.inf] = 0
    return log_likelihood

def make_data_and_recover_LUCE(p_alpha, p_beta, p_mu, pid, df):
    pairs = []  
    all_data = None 
    for i, amb in enumerate(ambiguities):
        subset_trials = np.random.random(size = n_trials_per_subgroup) 
        p_true = probability_of_lottery_choice_LUCE(p_alpha, p_beta, p_mu, lotteries[i], certain_value, amb, probabilities[i])
        idx = np.where(subset_trials <= p_true[0])
        sv_lott = p_true[1]
        sv_safe = p_true[2]
        subset_trials[:] = 0
        subset_trials[idx] = 1 ## error prone, real trials
        #subset_trials[:int(np.round(n_trials_per_subgroup * p_true))] = 1 ## perfect circumstances, no noise
        data = [pid, p_alpha, p_beta, p_mu, None, None, None, lotteries[i], certain_value, amb, probabilities[i], sv_lott, sv_safe]
        data = [[a] * n_trials_per_subgroup for a in data]
        data.append(subset_trials)
        data = np.array(data).T
        if all_data is None:
            all_data = data
        else:
            all_data = np.vstack((all_data, data))
        lottery_chosen_x = np.sum(subset_trials)
        pairs.append((lottery_chosen_x, n_trials_per_subgroup))
    res = MLE_alpha_mu_LUCE(np.array(pairs), lotteries, certain_value, ambiguities, probabilities)
    res = MLE_beta_LUCE(np.array(pairs), res[0], res[1], lotteries, certain_value, ambiguities, probabilities)
    df = pd_extend(df, pd.DataFrame(all_data, columns = df.columns))
    idx = np.where(df["Recovered Alpha"].values == None)
    df["Recovered Alpha"].values[idx] = res[0]
    df["Recovered Beta"].values[idx] = res[1]
    df["Recovered Mu"].values[idx] = res[2]
    return *res, df

def MLE_alpha_mu_LUCE(pairs, lotteries, certain_value, ambiguities, probabilities):
    best_fit = None
    max_likelihood = None
    y, n = pairs[:, 0], pairs[:, 1]
    idx = np.where(ambiguities == 0)
    for alpha in np.round(np.arange( 0.3, 2.01, 0.01), 2): ## grid search - part 1 (fit alpha first)
        for mu in np.round(np.arange(0.5, 0.71, 0.01), 2):
            likelihood = np.nansum(binomial_likelihood_LUCE(alpha, 0, mu, y[idx], n[idx], lotteries[idx], certain_value, ambiguities[idx], probabilities[idx]))
            if max_likelihood is None or likelihood > max_likelihood:
                max_likelihood = likelihood
                best_fit = (alpha, mu)
    return best_fit

def MLE_beta_LUCE(pairs, alpha, mu, lotteries, certain_value, ambiguities, probabilities):
    best_fit = None
    max_likelihood = None
    y, n = pairs[:, 0], pairs[:, 1]
    idx = np.where(ambiguities != 0)
    for beta in np.round(np.arange(-1.3, 1.31, 0.01), 2): ## grid search - part 2 (fit beta second)
        likelihood = np.nansum(binomial_likelihood_LUCE(alpha, beta, mu, y[idx], n[idx], lotteries[idx], certain_value, ambiguities[idx], probabilities[idx]))
        if max_likelihood is None or likelihood > max_likelihood:
            max_likelihood = likelihood
            best_fit = (alpha, beta, mu)
    return best_fit

def probability_of_lottery_choice_LUCE(alpha, beta, mu, lottery_value, certain_value, ambiguity, probability):
    SV_lottery, SV_certain = SVs(alpha, beta, lottery_value, certain_value, ambiguity, probability)
    return SV_lottery**(1 / mu) / (SV_lottery**(1 / mu) + SV_certain**(1 / mu)), SV_lottery, SV_certain

### **Luce #1**
- Simulating across beta and mu

In [ ]:
## Participant parameter space (~10,000 simulated subs)
alphas = np.array([0.6]) ## 1 step
betas  = np.round(np.arange(-1.3, 1.31, 2.6/498), 3) ## 500 steps
mus    = np.round(np.arange( 0.5,  0.7,    0.01), 2) ## 20 steps
print("Number of Simulations:", len(alphas)*len(betas)*len(mus))
df = pd.DataFrame()

df_participants = pd.DataFrame(columns=["PID", 
                                        "Simulated Alpha", "Simulated Beta", "Simulated Mu", 
                                        "Recovered Alpha", "Recovered Beta", "Recovered Mu",
                                        "Lottery Value", "Safe Value", 
                                        "Ambiguity", "Probability",
                                        "SV Lottery", "SV Safe", 
                                        "Choice"])
pid = 1
for p_mu in mus:
    for p_alpha in alphas:
        for p_beta in betas:
            r_alpha, r_beta, r_mu, df_participants = make_data_and_recover_LUCE(p_alpha, p_beta, p_mu, pid, df_participants)
            print(pid, end = "\r")
            pid += 1   
filename = os.path.join(save_dir, "simulation4LUCE_cAvBvM-2trials.csv")
df_participants.to_csv(filename)

In [ ]:
file = glob(os.path.join(base_proj_dir, save_dir, "simulation4LUCE_cAvBvM-2trials.csv"))[0]
df = pd.read_csv(file)
df2 = pd.DataFrame()

beta_x = df["Simulated Beta"].values[::total_trials]
fit_line = linregress(beta_x, df["Recovered Beta"].values[::total_trials])
fit_stat = spearmanr(beta_x, df["Recovered Beta"].values[::total_trials])
line_y = np.array(beta_x) * fit_line.slope + fit_line.intercept
residuals = np.array(beta_x) - df["Recovered Beta"].values[::total_trials]
print("Shapiro-Wilk Test of Normality:", shapiro(np.abs(residuals))[:])
print("Spearmans Rho^2:", fit_stat[:])
print("Residual SD:", np.std(residuals))
df2["Simulated β (Luce)"] = df["Simulated Beta"].values[::total_trials]
df2["Fit β"] = df["Recovered Beta"].values[::total_trials]
fig, ax = plt.subplots(figsize=(10, 10))
ax = sns.regplot(x = "Simulated β (Luce)", y = "Fit β", data = df2, color = "#D401B5")
sns.despine(top = True)
ax.set_ylabel("Fit β", fontsize = 16)
ax.set_xlabel("Simulated β", fontsize = 16)
plt.suptitle("n={5} (α=0.6, μ=[0.5,0.7,0.01]) | Shapiro-Wilk={0:.2f}, p={1:.2f}** | Spearmans Rho^2={2:.2f}, p={3:.2f}** | Residual SD={4:.2f}".format(shapiro(np.abs(residuals))[0], 
                                                                                                                                                        shapiro(np.abs(residuals))[1], 
                                                                                                                                                        fit_stat[0]**2, fit_stat[1], 
                                                                                                                                                        np.std(residuals), 
                                                                                                                                                        len(beta_x)), 
                                                                                                                                                        fontsize = 12)
fig_name = os.path.join(save_dir, "simulation4LUCE_cAvBvM-2trials.png")
print("Saving to: {}".format(fig_name))
plt.savefig(fig_name)

### **Luce #2**
- Simulating across alpha, beta, and mu

In [ ]:
## Participant parameter space (~180,000 simulated subs)
alphas = np.round(np.arange( 0.3,  1.2,    0.05), 2)
betas  = np.round(np.arange(-1.3, 1.31, 2.6/498), 3) ## 500 steps
mus    = np.round(np.arange( 0.5,  0.7,    0.01), 2) ## 20 steps
print("Number of Simulations:", len(alphas)*len(betas)*len(mus))

df = pd.DataFrame()
df_participants = pd.DataFrame(columns=["PID", 
                                        "Simulated Alpha", "Simulated Beta", "Simulated Mu", 
                                        "Recovered Alpha", "Recovered Beta", "Recovered Mu",
                                        "Lottery Value", "Safe Value", 
                                        "Ambiguity", "Probability",
                                        "SV Lottery", "SV Safe",
                                        "Choice"])
pid = 1
for p_mu in mus:
    for p_alpha in alphas:
        for p_beta in betas:
            r_alpha, r_beta, r_mu, df_participants = make_data_and_recover_LUCE(p_alpha, p_beta, p_mu, pid, df_participants)
            print(pid, end = "\r")
            pid += 1  
filename = os.path.join(save_dir, "simulation4LUCE_vAvBvM-2trials.csv")
df_participants.to_csv(filename)

In [ ]:
file = glob(os.path.join(base_proj_dir, save_dir, "simulation4LUCE_vAvBvM-2trials.csv"))[0]
df = pd.read_csv(file)
df2 = pd.DataFrame()

beta_x = df["Simulated Beta"].values[::total_trials]
fit_line = linregress(beta_x, df["Recovered Beta"].values[::total_trials])
fit_stat = spearmanr(beta_x, df["Recovered Beta"].values[::total_trials])
line_y = np.array(beta_x) * fit_line.slope + fit_line.intercept
residuals = np.array(beta_x) - df["Recovered Beta"].values[::total_trials]
print("Shapiro-Wilk Test of Normality:", shapiro(np.abs(residuals))[:])
print("Spearmans Rho^2:", fit_stat[:])
print("Residual SD:", np.std(residuals))
df2["Simulated β (Luce)"] = df["Simulated Beta"].values[::total_trials]
df2["Fit β"] = df["Recovered Beta"].values[::total_trials]
fig, ax = plt.subplots(figsize=(10, 10))
ax = sns.regplot(x = "Simulated β (Luce)", y = "Fit β", data = df2, color = "#E15930")
sns.despine(top = True)
ax.set_ylabel("Fit β", fontsize = 16)
ax.set_xlabel("Simulated β (Luce)", fontsize = 16)
plt.suptitle("n={5} (α[0.3,1.2,0.05], μ=[0.5,0.7,0.01]) | Shapiro-Wilk={0:.2f}, p={1:.2f}** | Spearmans Rho^2={2:.2f}, p={3:.2f}** | Residual SD={4:.2f}".format(shapiro(np.abs(residuals))[0], 
                                                                                                                                                                shapiro(np.abs(residuals))[1], 
                                                                                                                                                                fit_stat[0], fit_stat[1], 
                                                                                                                                                                np.std(residuals)), 
                                                                                                                                                                len(beta_x), 
                                                                                                                                                                fontsize = 12)
fig_name = os.path.join(save_dir, "simulation4LUCE_vAvBvM-2trials.png")
print("Saving to: {}".format(fig_name))
plt.savefig(fig_name)